
### Using a Query Engine to Answer Queries

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20Mastering_Advanced_RAG.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

A  [Query Engine](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/root.html)  is an advanced interface that allows interaction with data via natural language queries. It’s a wrapper designed to process queries and generate responses. Combining multiple query engines can enhance functionality, meeting the complexity of specific queries.

On the other hand, a  [Chat Engine](https://docs.llamaindex.ai/en/stable/module_guides/deploying/chat_engines/root.html)  is suitable for an interactive experience like a conversation, as it requires a series of queries and responses. This offers a more dynamic and engaging way to interact with data.

To create a query engine, one typically uses the `.as_query_engine()` method on generated indices. Here are the steps included in creating indexes from text files and using query engines to engage with the dataset:

Install necessary packages using the command: `!pip install -q llama-index==0.9.14.post3 deeplake==3.8.8 openai==1.3.8 cohere==4.37.`

And set up the API key environment variables:

In [ ]:
import os
from advanced_rag_custom_utils.helper import get_openai_api_key, get_activeloop_api_key, get_cohere_api_key
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()
COHERE_API_KEY = get_cohere_api_key()

Download a text file as your source document. We used a file containing a collection of essays by Paul Graham consolidated into a single text file. You can also download this file directly from [towardsai.net/book](http://towardsai.net/book). Alternatively, you can use the following commands in your terminal to create a directory and download the file into it:

In [ ]:
!mkdir -p './paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O './paul_graham/paul_graham_essay.txt'

Now, use the `SimpleDirectoryReader` in the LlamaIndex framework to read all files from the designated directory. This class is designed to automatically navigate through the files, converting them into Document objects.

In [ ]:
from llama_index import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("./paul_graham").load_data()

We will also use the `ServiceContext` to break the lengthy single document into numerous smaller chunks with some overlap. Following that, we will make nodes from the generated documents.

In [ ]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=512, chunk_overlap=64)
node_parser = service_context.node_parser

nodes = node_parser.get_nodes_from_documents(documents)

The nodes should be stored in a vector database for convenient access. The `DeepLakeVectorStore` class can create an empty dataset by specifying a path. You can access the processed dataset using the genai360 organization ID or update it to match your *Activeloop* username and store the data on your account.

In [ ]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = ACTIVELOOP_API_KEY
my_activeloop_dataset_name = "LlamaIndex_paulgraham_essays"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

The new database will be used within a `StorageContext` object, allowing for the processing of nodes to establish relationships as required. At last, the `VectorStoreIndex`  receives the nodes and their corresponding links to the database and uploads the data to the cloud. It builds the index and creates embeddings for each segment.

In [ ]:
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.docstore.add_documents(nodes)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

The generated index is the base for defining the query engine. To start a query engine, you can utilize the index object and call the `.as_query_engine()` method. The code snippet below uses the streaming flag to improve the end user’s experience by minimizing idle waiting time (further information will be provided on this topic). In addition, it utilizes the `similarity_top_k` flag to determine the number of documents to retrieve from the index to answer the query.

In [ ]:
query_engine = vector_index.as_query_engine(streaming=True, similarity_top_k=10)

The final step is interacting with the source data using the `.query()` method. We can now ask questions, and the query engine generates answers using retrievers and a response synthesizer.

In [ ]:
streaming_response = query_engine.query(
    "What does Paul Graham do?",
)
streaming_response.print_response_stream()

    Paul Graham is an artist and entrepreneur. He is passionate about creating paintings that can stand the test of time. He has also co-founded Y Combinator, a startup accelerator, and is actively involved in the startup ecosystem. While he has a background in computer science and has worked on software development projects, his primary focus is on his artistic pursuits and supporting startups.

The query engine can be set up to operate in a streaming mode, delivering a response stream in real-time to improve interactivity. This feature is advantageous in minimizing downtime for end users. Users can easily view each word as it is generated, eliminating the need to wait for the model to produce the complete response.

### Splitting Complex Queries into Subqueries

The `SubQuestionQueryEngine` is a class designed to handle complex queries effectively. This engine can break down a user’s primary question into multiple sub-questions, address each individually, and subsequently synthesize the answers to formulate a comprehensive response. To implement this approach, alter the earlier query engine configuration, specifically deactivate the streaming flag, as it is incompatible with this method.

In [ ]:
query_engine = vector_index.as_query_engine(similarity_top_k=10)

The `query_engine` can be registered as a tool by using the `QueryEngineTool` class, along with descriptive metadata. This description informs the framework about the functionalities of this tool, facilitating the selection of the most appropriate tool for a given task, particularly in scenarios where there are multiple tools at disposal. Following this, the integration of previously declared tools and the service context, as established earlier, is utilized to initialize the `SubQuestionQueryEngine` object.

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="pg_essay",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

The pipeline is ready to split a question into simpler subquestions. As shown, it formulates three queries, each responding to a portion of the query, and attempts to locate their answers separately. A response synthesizer then processes the responses to produce the final output.

In [ ]:
response = query_engine.query(
    "How was Paul Grahams life different before, during, and after YC?"
)
print( ">>> The final response:\n", response )

 >>> The final response:  
Paul Graham's life was different before, during, and after YC. Before YC, he worked on a variety of projects including writing essays, developing YC's internal software in Arc, and creating Hacker News. During YC, his focus shifted to writing essays and working on YC itself. After YC, he continued writing essays but also worked on various projects such as developing the programming language Arc and later its new version called Bel. He also explored other potential projects and engaged in painting for a period of time. Overall, his work and interests evolved throughout these different phases of his life.

### Customizing a Retriever Engine

The retriever and its settings, such as the number of returned documents, significantly affect the quality and relevance of the `QueryEngine’s` results. LlamaIndex enables the creation of custom retrievers that blend various styles, optimizing retrieval strategies for specific queries. For example, the `RetrieverQueryEngine` operates with a selected retriever that is used to shape query answers. Two main retriever types to use with it are: `VectorIndexRetriever`, focusing on the top-k most relevant nodes for precision, ideal for detailed research, and `SummaryIndexRetriever`, which compiles all related nodes, providing a broad overview suitable for exploratory searches where exhaustive information is more valuable than exact relevance.

>💡You can read the usage example tutorial: [Building and Advanced Fusion Retriever from Scratch](https://docs.llamaindex.ai/en/stable/examples/low_level/fusion_retriever.html) at [towardsai.net/book](http://towardsai.net/book).

## Reranking Documents

While retrieval mechanisms capable of extracting multiple segments from long documents are generally efficient, they sometimes include irrelevant results. Reranking involves re-evaluating and reordering search results to highlight the most relevant options. By discarding segments with lower relevance scores, the final context provided to the LLM is more focused.

The ***Cohere Reranker*** improves the retrieval accuracy of closely related content. Although the semantic search component is proficient at sourcing relevant documents, the  [Rerank endpoint](https://cohere.com/rerank?ref=txt.cohere.com)  enhances the quality of these results, particularly for complex and domain-specific queries. It rearranges the search outcomes based on their relevance to the query. It is essential to understand that Rerank is not a substitute for a search engine but an additional and complementary tool that optimizes the ordering of search results for the most practical user experience.

The reranking process begins by organizing documents into batches. Subsequently, the LLM evaluates each batch, assigning relevance scores to them. The reranking process concludes with the compilation of the most relevant documents from all these batches to create the final retrieval response. This approach ensures that the most relevant information is emphasized and central to the search results. Here is a code example using a reranker.

Install all the necessary packages; acquire your API key from  [Cohere.com](http://cohere.com/)  and replace the provided placeholder with this key.

In [ ]:
import cohere

# Get your cohere API key on: www.cohere.com
co = cohere.Client(COHERE_API_KEY)

# Example query and passages
query = "What is the capital of the United States?"
documents = [
   """Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.""",
   """The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.""",
   """Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.""",
   """Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. """,
   """Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.""",
   """North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of government is Bismarck."""
]

A rerank object is created by passing the query and the documents. Additionally, the argument `rerank_top_k` is set to `3,` directing the system to select the top three highest-scored candidates as determined by the model. The model used for reranking in this instance is *rerank-multilingual-v2.0.*

In [ ]:
results = co.rerank(query=query, documents=documents, top_n=3,
model='rerank-english-v2.0') # Change top_n to change the number of
# results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Document: {r.document['text']}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

    **Document Rank: 1**, Document Index: 3  
    Document: Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.  
    Relevance Score: 0.99  
      
    **Document Rank: 2**, Document Index: 1  
    Document: The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.  
    Relevance Score: 0.30  
      
    **Document Rank: 3**, Document Index: 5  
    Document: Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment.  
    Relevance Score: 0.27


This task can be accomplished by combining LlamaIndex with Cohere Rerank. The rerank object can be integrated into a query engine, allowing it to efficiently manage the reranking process in the background. We will use the same vector index defined earlier to avoid duplicating code. The CohereRerank class creates a rerank object, requiring the API key and specifying the number of documents to be returned after the scoring process.

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank(api_key=COHERE_API_KEY, top_n=2)

The combination of the `as_query_engine` method and the `node_postprocessing` argument can be used to integrate the reranker object. In this setup, the retriever selects the top 10 documents based on *semantic similarity*. Afterward, the reranker refines this selection, narrowing it down to the two most relevant documents.

In [ ]:
query_engine = vector_index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)

response = query_engine.query(
    "What did Sam Altman do in this essay?",
)
print(response)

    Sam Altman was asked if he wanted to be the president of Y Combinator (YC) and initially said no. However, after persistent persuasion, he eventually agreed to take over as president starting with the winter 2014 batch.


>💡Rerank computes a relevance score for the query and each document and returns a sorted list from the most to the least relevant document.

The reranking process in search systems improves the quality and precision of the returned results. It enhances existing systems without requiring comprehensive modifications, offering a cost-efficient approach to improve search functionality. Cohere Rerank is a very popular choice for a reranker that can easily be used via API.

## Recursive Retrieval and Small-To-Big Retrieval

An alternative approach to retrieving relevant documents is to use document summaries instead of extracting fragmented snippets or brief text chunks to respond to queries. This method ensures that the responses encapsulate the full context or topic under consideration, providing a more comprehensive understanding of the subject. It includes two types of retrieval: recursive and small-to-big retrieval.

Recursive retrieval is particularly effective for documents with a hierarchical structure, as it facilitates forming links and connections between nodes. Jerry Liu, the founder of LlamaIndex, highlights its applicability in situations like a PDF file, which might contain “sub-data” such as tables, diagrams, and references to other documents. This method efficiently navigates through the graph of interconnected nodes to pinpoint information. Its versatility allows for its use in various contexts, including node references, document agents, and query engines. For practical implementations, such as processing a PDF file and extracting data from tables, the LlamaIndex  [Recursive Retriever tutorials](https://docs.llamaindex.ai/en/stable/examples/query_engine/pdf_tables/recursive_retriever/)  are accessible at  [towardsai.net/book](http://towardsai.net/book).

The small-to-big retrieval approach enhances the effectiveness of RAG pipelines by separating the text chunks used for retrieval from those used for synthesis. Instead of embedding and retrieving large text chunks that may contain filler content and obscure semantic relevance, this approach utilizes smaller, more targeted chunks for retrieval. After identifying relevant smaller chunks, the associated larger chunks are used to provide broader context for synthesis. This method improves retrieval accuracy and ensures the LLM has sufficient context. Techniques include using smaller child chunks linked to bigger parent chunks and sentence window retrieval.

Find a hands-on tutorial on  [implementing Small-to-Big Retrieval](https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/MetadataReplacementDemo/)  from the documentation at  [towardsai.net/book](http://towardsai.net/book).